# Power Outages
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the severity (number of customers, duration, or demand loss) of a major power outage.
    * Predict the cause of a major power outage.
    * Predict the number and/or severity of major power outages in the year 2020.
    * Predict the electricity consumption of an area.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
In this project, we will be using a classification prediction model to predict, based on given data, if an outage will or will not be severe. For the purposes of this project, we define severity in terms of outage duration, and use a Decision Tree Classifer as the model. This means a longer outage duration will indicate a more severe outage. Of course, we could have defined severity by the number of people affected, or another combination of our given data, but for this purpose we are sticking with outage duration. In order to define an outcome, we set 1 equal to a severe outage and 0 equal to a non severe outage using a Binarizer with a threshhold at the mean outage duration. This, of course, is our target variable. We run a basic baseline model only one-hot-encoding features that need to be, and remove columns that have no affect on the outcome of the data (such as POSTAL.CODE, which is a repeat of STATE). This baseline model has an accuracy of roughly 77%. Then, we build our final model by adding features not present in the dataset, such as the season of the outage. We go more into detail in the "Final Model" section. Additionally, we edit our Decision Tree Classifier parameters to get the best outcome that we can. Lastly, we run a fairness evaluation on our model to assess how fair the model accuracy is. Our objective is to increase our accuracy as much as possible. 

It is important to note that we only have 1540 rows of data over years 2000 to 2016, meaning that it is not valid to say that this data would accurately predict data, for lets say, 2020. This model could be improved with access to more data. 
### Baseline Model
In our baseline model, we use 23 features to predict if the outage is severe (1) or not (0). We use a Binarizer to create an "Outcome" column for our classifier model, as stated above. We fill null values in the Outage Duration column with 0, due to results from Project03 that showed null values in duration were often due to intentional attacks and the outage was not long. All other columns were removed due to not having any affect on the model. Below are the labels for each column and if it is nominal, ordinal, or quantitative: 
'MONTH': N 
'U.S._STATE': N
'NERC.REGION': N
'CLIMATE.REGION': N
'CAUSE.CATEGORY.DETAIL': N
'DEMAND.LOSS.MW': Q
'CUSTOMERS.AFFECTED': Q
'TOTAL.PRICE': Q
'RES.SALES': Q
'COM.SALES': Q
'IND.SALES': Q
'TOTAL.SALES': Q
'IND.PERCEN': Q
'RES.CUSTOMERS': Q
'IND.CUSTOMERS': Q
'TOTAL.CUSTOMERS': Q
'RES.CUST.PCT': Q
'IND.CUST.PCT': Q
'PC.REALGSP.REL':Q
'PC.REALGSP.CHANGE': Q
'TOTAL.REALGSP': Q
'POPULATION': Q
'Outcome': Ordinal or Nominal?  
Our baseline model first does some data cleaning to convert columns that are currently objects into floats. Then, we filter through the columns and add a step to the pipeline to one hot encode any categorical column. We create a column transformer and feed our column transformer into a Pipeline with a Decision Tree Classifier. With X as our predictor variabled and y as our target, we fit and calculate the score which gives the accuracy percent of the model. Baseline, we get 77% accuracy. Overall, this is not a great predictor and can definitely be improved by adding features. 

### Final Model
In our final model, we begin by showing, by example, how generalizing a feature can lead to a better predictor. In this data, there are two columns detailing the category of the outage. One contains more detailed descriptions. In our baseline, we use the detailed descriptions to show how getting too specific can lead to overfitting data and not having a generalized enough model. As a solution, we can categorize similar category details into one category. This is already done in our dataset and is named category, so we drop the details and add in the generalized category. When we re-run our model, we get a 82% accuracy. This increased! Next, we know that seasons are associated with storms that are more likely to set off a severe outage. Thus, we group 'MONTH' into four seasons: winter, spring, summer, and autumn. This means we are generalizing our model to make better predicitions based off the season an outage occured in. We use this column in addition to the orginial ones from the baseline model and increase our accuracy to 86.5%! Lastly, we noticed a quantiative column that had very large values and were a little hard to compare, leading us to believe a log-scale would make comparisons easier, and lead to an overall better model. For the last step, we log-scale the TOTAL.REALGSP column and run the Decision Tree Classifier model again, with a max_depth set to 25 for best parameter, found from grid search. This gives us a 93.6% accuracy overall!

### Fairness Evaluation
To evaluate the fairness of the final data model we decided to do a permutation test to find accuracy parity utilizing the difference in score between two randomly selected subsets of the data as our test statistic. Our null hypothesis was that the model was not biased in terms of predicting outage severity. Our alternative hypothesis was that the model was not fair. We decided to shuffle the population column and split up our data based on values above and below the median. We chose to utilize the population column to split up our data because it had no null values. Once we got the subsets of the data we ran our final model on each subset of the data and found the score of the model. Then we returned this absolute difference of score as the test statistic. We ran this proccess of getting the test statistic 1000 times to get a distribution of test statistics. We found our p-value based on the proportion of test statistics above a pre-determined threshold. We chose our threshold to be .02 so their was some room for error on the score since our final model was not perfect with a score of around .94. In the end we got a p-value of .02 so we were able to determine that our model was fair at signifigance level of .05.

# Code

In [476]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning/Prep
This section is the exact same as project03, where we clean the dataset with the appropriate measures needed to create models.

In [490]:
#Read in the data frame
df = pd.read_csv('outage.csv')
data = df.copy()

In [491]:
#CLEANS THE DF INTO A USEABLE FORMAT
#grab the columns from the messy dataframe
df.columns = df.loc[4]
#grab the units
units = df.loc[5][1:]
#drop unnessessary columns and reset the index
df = df.drop([0,1,2,3,4,5]).drop(columns = ['variables']).reset_index(drop = True)

In [492]:
#FIXING COLUMNS
#combine the start and restoration date and times 
df['OUTAGE.START'] = df['OUTAGE.START.DATE']+" "+df['OUTAGE.START.TIME']
df['OUTAGE.RESTORATION'] = df['OUTAGE.RESTORATION.DATE']+" "+df['OUTAGE.RESTORATION.TIME']

#Converting to datetime object
df['OUTAGE.START'] = pd.to_datetime(df['OUTAGE.START'])
df['OUTAGE.RESTORATION'] = pd.to_datetime(df['OUTAGE.RESTORATION'])
df['OUTAGE.START.DATE'] = pd.to_datetime(df['OUTAGE.START.DATE'])
df['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE'])
df['OUTAGE.START.TIME'] = pd.to_datetime(df['OUTAGE.START.TIME'])
df['OUTAGE.RESTORATION.TIME'] = pd.to_datetime(df['OUTAGE.RESTORATION.TIME'])
#We can now drop the orginial columns 
#sets the index as observatins column
df = df.set_index('OBS')
#convert datapoints to a float
def convert(x):
    if x != x:
        return x
    try:
        out = float(x)
    except:
        out = x
    return out
#df = df.applymap(convert)
df = df[df['OUTAGE.START'].isnull() == False]
df = df[df['CLIMATE.REGION'].isnull() == False]

### Baseline Model

Below, we will start to build a model to predict if an outage will be severe. For the purposes of this project, we define severe based off the duration of the outage. If the duration is above the mean outage duration, it will be assigned 1 for severse, else it will be assigned 0 for not severe. We do this using the Binarizer with a threshold set to the mean outage duration in this dataset. We use a fillna value of zero because in our last project, we found NaN values typically meant that the outage was short and unreported

In [493]:
from sklearn.preprocessing import Binarizer 
#we fill with 0 because these are usually intentional attacks
df[['OUTAGE.DURATION']] = df[['OUTAGE.DURATION']].fillna(value = 0)

#create a binarizer
transformer = Binarizer(threshold = 2550).fit(df[['OUTAGE.DURATION']])
#create outcomes based on data
outcomes = transformer.transform(df[['OUTAGE.DURATION']])
#define the outcomes
df['Outcome'] = outcomes
display(df[['OUTAGE.DURATION','Outcome']].head(5))

4,OUTAGE.DURATION,Outcome
OBS,,
1,3060,1.0
2,1,0.0
3,3000,1.0
4,2550,0.0
5,1740,0.0


Here, we will create our baseline model. This model is pretty basic, only one hot encoding state. We first drop columns with no relationship to the severity prediction. This was done by analyzing relationships between the accuracy with and without the column. Then, we do a little data cleaning with a convert function to turn our data into numerical data. We then grab our X and y variables and check which columns need to be one hot encoded (such as state). After all of this, we stick it in a Column Transformer and Pipeline using a Decision Tree Classifier. This gives us an accuracy of around 77%. 

In [494]:
def run_model(data):
    #convert items to float
    d = data.copy()
    def convert(x):
        if x != x:
            return x
        try:
            out = float(x)
        except:
            out = x
        return out
    d = d.applymap(convert)
    steps = []
    X = d.drop('Outcome', axis=1)
    y = d['Outcome']
    #the code below one hot encodes any string column
    for col in d.columns:
        typ = type(d[col][0])
        if typ == str:
            steps.append((col,OneHotEncoder(handle_unknown='ignore'),[col]))

    col_t = ('col_trans',ColumnTransformer(steps))
    pl = Pipeline([col_t,('tree', DecisionTreeClassifier())])

    fitted = pl.fit(X, y)
    accuracy = pl.score(X, y)
    return pl, accuracy

In [495]:
#drop cols with no relationship
data = df.drop(columns = ['YEAR','POSTAL.CODE','OUTAGE.START.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME',
                  'OUTAGE.RESTORATION.DATE','CAUSE.CATEGORY','HURRICANE.NAMES','POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL','OUTAGE.START','OUTAGE.RESTORATION', 'ANOMALY.LEVEL',
                         'RES.PRICE','COM.CUSTOMERS','COM.CUST.PCT','PC.REALGSP.USA',
                         'UTIL.REALGSP','AREAPCT_UC','IND.PRICE','PCT_WATER_TOT','CLIMATE.CATEGORY',
                         'PC.REALGSP.STATE','AREAPCT_URBAN','COM.PRICE','PI.UTIL.OFUSA',
                          'PCT_LAND','PCT_WATER_INLAND','RES.PERCEN','COM.PERCEN','UTIL.CONTRI', 'OUTAGE.DURATION'])
pl, acc = run_model(data)
acc

0.7743421052631579

We can use the below function to see which features are most important to our dataset. Customers affected, for example, has a large affect on the outcome, as well as the NERC region of the outage, population, and total real GSP contributed by all industries. We can use this information to extract new features that may help our accuracy in the final model.

In [496]:
dec_tree = pl['tree']
#We can use the dictionary to see what cols are necessary! 
dict(zip(X.columns, dec_tree.feature_importances_))

{'MONTH': 0.0001506536519306383,
 'U.S._STATE': 0.00018379674057494776,
 'NERC.REGION': 0.009824450414607656,
 'CLIMATE.REGION': 1.9761886119325722e-05,
 'CAUSE.CATEGORY': 0.0,
 'OUTAGE.DURATION': 0.005503415887121589,
 'DEMAND.LOSS.MW': 0.028287781270694267,
 'CUSTOMERS.AFFECTED': 0.00017582212434864955,
 'TOTAL.PRICE': 0.0005781809330629655,
 'RES.SALES': 0.0015168466536754615,
 'COM.SALES': 0.002625401259704883,
 'IND.SALES': 9.761385602670174e-05,
 'TOTAL.SALES': 0.008679519136865356,
 'IND.PERCEN': 0.006117031174440262,
 'RES.CUSTOMERS': 0.004434647756593718,
 'IND.CUSTOMERS': 0.006798286501802084,
 'TOTAL.CUSTOMERS': 0.004924339407073903,
 'RES.CUST.PCT': 1.1691002465366075e-05,
 'IND.CUST.PCT': 0.0,
 'PC.REALGSP.REL': 0.0,
 'PC.REALGSP.CHANGE': 0.31588040855315874,
 'TOTAL.REALGSP': 0.014462762871960608,
 'POPULATION': 0.0,
 'SEASON': 0.009906151042612723}

In this baseline model, we have a max tree depth of 31 and 139 number of nodes. 

In [497]:
print("Max tree depth: "),print(dec_tree.tree_.max_depth)
print("Number of nodes: "),print(dec_tree.tree_.node_count)

Max tree depth: 
31
Number of nodes: 
139


(None, None)

### Final Model

One of the first steps we can take is to generalize our data better. For example, we can change CAUSE.CATEGORY.DETAIL into just CAUSE.CATEGORY, grouping similar outage types together. Luckily, this is already done for us in the dataset. After adding this change, accuracy goes from 77% to 82%. 

In [498]:
final_data1 = df.drop(columns = ['YEAR','POSTAL.CODE','OUTAGE.START.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME',
                  'OUTAGE.RESTORATION.DATE','CAUSE.CATEGORY.DETAIL','HURRICANE.NAMES','POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL','OUTAGE.START','OUTAGE.RESTORATION', 'ANOMALY.LEVEL',
                         'RES.PRICE','COM.CUSTOMERS','COM.CUST.PCT','PC.REALGSP.USA',
                         'UTIL.REALGSP','AREAPCT_UC','IND.PRICE','PCT_WATER_TOT','CLIMATE.CATEGORY',
                         'PC.REALGSP.STATE','AREAPCT_URBAN','COM.PRICE','PI.UTIL.OFUSA',
                          'PCT_LAND','PCT_WATER_INLAND','RES.PERCEN','COM.PERCEN','UTIL.CONTRI'])


pl, acc = run_model(final_data1)
acc

0.8289473684210527

We can add more features to our dataset to get a better model. When it comes to power outages,
seasons may categorize the chance of a severe outage better. For example, winter ice storms and summer thunderstorms may have a higher impact than the occasional spring shower. With this assumption, we can categorize our MONTH column into four seasons: Spring, Summer, Autumn, and Winter. Then, we can one hot encode the Seasons column and run the model again. After completing this step, our accuracy jumps 4% to 86.5%!

In [499]:
#The seasons are defined as:
#spring (March, April, May), 
#summer (June, July, August), 
#autumn (September, October, November) and 
#winter (December, January, February).

#Create a replacement dictionary 
season_dict = {1:'Winter', 2:'Winter',3:'Spring',4:'Spring',5:'Spring',
              6:'Summer',7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',
              11:'Autumn',12:'Winter'}

#drop irrelevant columns 
final_data2 = df.drop(columns = ['YEAR','POSTAL.CODE','OUTAGE.START.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME',
                  'OUTAGE.RESTORATION.DATE','CAUSE.CATEGORY.DETAIL','HURRICANE.NAMES','POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL','OUTAGE.START','OUTAGE.RESTORATION', 'ANOMALY.LEVEL',
                         'RES.PRICE','COM.CUSTOMERS','COM.CUST.PCT','PC.REALGSP.USA',
                         'UTIL.REALGSP','AREAPCT_UC','IND.PRICE','PCT_WATER_TOT','CLIMATE.CATEGORY',
                         'PC.REALGSP.STATE','AREAPCT_URBAN','COM.PRICE','PI.UTIL.OFUSA',
                          'PCT_LAND','PCT_WATER_INLAND','RES.PERCEN','COM.PERCEN','UTIL.CONTRI'])
# convert items to float (we do this outside because month needs to be a float)
def convert(x):
    if x != x:
        return x
    try:
        out = float(x)
    except:
        out = x
    return out
final_data2 = final_data2.applymap(convert)

# replace the months with season and add a column to the dataset
final_data2['SEASON'] = final_data2['MONTH'].replace(season_dict)
# run the model 
pl, acc = run_model(final_data2)
acc

0.8644736842105263

Now that we have an 86% accuracy, we can create another feature to increase our accuracy again. To do this, we reference our dictionary of important features from the baseline model section. There, we see that 'TOTAL.REALGSP' is considered one of the most important features in this model. Below, we check out some basic stats on this specific column. Since it seems there is a skewness in the data with large values, and a decently large standard deviation, we can log-scale this column to have our values more equally spaced. This, in return, should help in comparisons when creating our model and lead to a higher accuracy.


In [500]:
final_data2[['TOTAL.REALGSP']].describe()

4,TOTAL.REALGSP
count,1.520000e+03
mean,6.721794e+05
std,6.290742e+05
min,2.681100e+04
25%,2.497585e+05
50%,4.074030e+05
75%,1.071033e+06
max,2.317466e+06


Below, our final model gets a 93.6% accuracy as we expected! Now, the last step is to mess with the parameters of our Decision Tree Classifier. When we run a quick grid search, we find that having a max_depth of 25 helps our model the most, and keeping a default on other parameters is best. This means we have gone from a 77% baseline model to a 93.6% final model!  

In [501]:
def run_final_model(data):
    #convert items to float
    d = data.copy()
    def convert(x):
        if x != x:
            return x
        try:
            out = float(x)
        except:
            out = x
        return out
    d = d.applymap(convert)
    steps = [('log-scale',FunctionTransformer(np.log),['TOTAL.REALGSP'])]
    X = d.drop('Outcome', axis=1)
    y = d['Outcome']
    #the code below one hot encodes any string column
    for col in d.columns:
        typ = type(d[col][0])
        if typ == str:
            steps.append((col,OneHotEncoder(handle_unknown='ignore'),[col]))

    col_t = ('col_trans',ColumnTransformer(steps))
    pl = Pipeline([col_t,('tree', DecisionTreeClassifier(max_depth = 25))])

    fitted = pl.fit(X, y)
    accuracy = pl.score(X, y)
    return pl, accuracy
final_pl, final_acc = run_final_model(final_data2)
final_acc

0.9368421052631579

### Fairness Evaluation

In [507]:
def get_t_stat(data):
    '''Function to get a t_stat for difference of scores between 2 random subsets of the data'''
    #Create copy of the data so original df is not manipulated
    df = data.copy()
    #Shuffles the column 'POPULATION'
    df['POPULATION'] = np.random.permutation(df['POPULATION'].values)
    #Split into 2 subsets of the data to do permutation test based on whether or not the values of 'POPULATION'
    #based on whether they are above or below median.
    temp = data.copy()
    median = temp['POPULATION'].sort_values()[df.shape[0] // 2]
    df1 = df[df['POPULATION'] >= median]
    df2 = df[df['POPULATION'] < median]
    #compare the absolute difference of score for each subset as my t_stat
    pl1, score1 = run_final_model(df1)
    pl2, score2 = run_final_model(df2)
    t_stat = abs(score1 - score2)
    return t_stat
t_stats = []
#Create a distribution of N t_stats
N = 1000
for i in range(N):
    t_stats.append(get_t_stat(final_data2))
#Get p_value based on amount of t_stats over a predetermined threshold
threshold = .02
p_value = np.count_nonzero(np.array(t_stats) > threshold) / final_data2.shape[0]
p_value

0.025657894736842105